In [ ]:
import numpy as np
import pandas as pd
import chainer
import chainer.links as L
import chainer.functions as F
df = pd.read_csv('wine_class.csv')

In [ ]:
df.head()

## 入力変数と教師データ（出力変数）に切り分ける

In [ ]:
t = df.iloc[:, 0:1]
t.head()

In [ ]:
input_x = df.iloc[:,1:]
input_x.head()

In [ ]:
print(type(t))
print(type(input_x))

In [ ]:
print(input_x.shape)

In [ ]:
type(input_x.values)#If add values, type transfrom to numpy.ndarray

In [ ]:
t = t.values - 1
input_x = input_x.values

In [ ]:
print(t.dtype)
print(input_x.dtype)

In [ ]:
t = t.astype('i')
input_x = input_x.astype('f')

In [ ]:
print(t.dtype, input_x.dtype)

In [ ]:
class NN(chainer.Chain):
    
    def __init__(self, n_mid_units = 5, n_out = 3):
        super().__init__()
        with self.init_scope():
            self.fc1 = L.Linear(None, n_mid_units)
            self.fc2 = L.Linear(None, n_out)
            
    def __call__(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        return h
        

In [ ]:
 #define the model
np.random.seed(1)
nn = NN()
model = L.Classifier(nn)

In [ ]:
dataset = list(zip(input_x, t))

In [ ]:
int(len(dataset)*0.7)

In [ ]:
n_train = int(len(dataset)*0.7)

In [ ]:
train, test = chainer.datasets.split_dataset_random(dataset, n_train, seed = 1)

In [ ]:
optimizer = chainer.optimizers.SGD()#確率的勾配降下法

In [ ]:
optimizer.setup(model)

## ミニバッチ学習
* バッチサイズ→100個だとしたらデータを100個ごとに分ける。
パラメータの更新
* 1epoch→10,00iteration

In [ ]:
batchsize = 10

In [ ]:
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize,repeat=False)

In [ ]:
from chainer import training

In [ ]:
updater = training.StandardUpdater(train_iter, optimizer, device=-1)#GPUとCPUどっちクを使うか。-1の場合CPU、０の場合GPU

In [ ]:
from chainer.training import extensions

In [ ]:
epoch = 50

In [ ]:
#trainerの宣言
trainer = training.Trainer(updater, (epoch, 'epoch'), out='result/wine')

In [ ]:
#検証データで評価
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))

In [ ]:
trainer.extend(extensions.LogReport(trigger=(1, 'epoch')))
#1エポックご(trigger)にtrainデータに対するlos.accuarcyとtestデータに対するlos/accuracys,経過時間を出力
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy', 'main/loss', 'validationmain/loss', 'elapsed_time']), trigger=(2,'epoch'))

In [ ]:
trainer.run()